In [ ]:
# 📦 Imports
import numpy as np

In [ ]:
# 🔌 Step 1: Connect to Milvus
from src.vector_database import get_client

client = get_client()
print("✅ Connected to Milvus")

In [ ]:
# 📁 Step 2: Define and Create Collection
collection_name = "demo_milvusclient"

# Drop if it already exists
if client.has_collection(collection_name):
    client.drop_collection(collection_name)
    print(f"⚠️ Dropped existing collection: {collection_name}")


client.create_collection(collection_name, dimension=128)
print(f"✅ Created collection: {collection_name}")

In [ ]:
# 🧠 Step 3: Generate and Insert Data
np.random.seed(42)
vectors = [list(np.random.rand(128).astype(np.float32)) for _ in range(100)]
ids = list(range(100))

# Insert records
insert_data = [{"id": ids[i], "vector": vectors[i]} for i in range(100)]
client.insert(collection_name=collection_name, data=insert_data)
print("✅ Inserted 100 vectors")

In [ ]:
# ⚙️ Step 4: Create Index

idx_params = client.prepare_index_params()
idx_params.add_index(
    field_name="vector",
    index_type="IVF_FLAT",
    metric_type="L2",
    params={"nlist": 64})

client.list_indexes(collection_name)


client.create_index(
    collection_name,
    index_params=idx_params
)
print("✅ Created IVF_FLAT index on 'vector'")

In [ ]:
# 🚀 Step 5: Load Collection
client.load_collection(collection_name)
print("✅ Loaded collection into memory")

In [ ]:
# 🔍 Step 6: Search Similar Vectors
search_vector = [vectors[0]]  # Search using the first vector

results = client.search(
    collection_name=collection_name,
    data=search_vector,
    anns_field="vector",
    limit=5,
    params={"nprobe": 10},
    output_fields=["id"]
)

print("\n🔎 Top 5 Similar Vectors:")
for hit in results[0]:
    print(f"ID: {hit['id']}, Distance: {hit['distance']:.4f}")

In [ ]:
# ❌ Step 7: Delete a Record
client.delete(collection_name, filter="id in [0]")
print("🗑️ Deleted record with ID 0")

In [ ]:
# 🧹 Step 8: Drop Collection
client.drop_collection(collection_name)
print(f"🧼 Dropped collection: {collection_name}")